In [2]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os

#### Human and natural classification (Kras, 2019)

In [3]:
path_kras = r'P:\1000545-054-globalbeaches\00_Global_Hindcasted_Shorelines\06_Drivers_Kras'

df_hotspot_acc = pd.read_csv(os.path.join(path_kras, 'hot_acc_inc_cluster.csv'))

df_hotspot_acc.head(2)

,continent,country,country_id,spatial_scale,width_range,min_spatial_signature,max_spatial_signature,spatial_magnitude_range,perc_larg_cr,perc_extr_cr,...,naminmean,namaxstds,naminstds,namaxbfit,naminbfit,nalinfval1,nalinfval2,nalinfstd1,nalinfstd2,cluster_id
0,Europe,Norway,NOR,2.5,3.3,0.4,1.0,1.2,80.0,20.0,...,-66.075438,57.697578,-43.290151,9.350650,-47.018079,1.686576,2.975714,0.457793,8.523640,2
1,Europe,Norway,NOR,2.5,2.1,2.5,5.0,1.4,60.0,40.0,...,-51.344686,57.606149,-16.085220,89.732411,-7.824512,8.213295,278.548863,1.774665,32.017823,0


#### Single Drivers (van der Heijden, 2023)

In [4]:
df_drivers = pd.read_csv('00. data/all_drivers.csv', index_col = False)
df_drivers = df_drivers[[c for c in df_drivers.columns if not 'Unnamed' in c]]
df_drivers.columns = map(str.lower, df_drivers.columns)
df_drivers = df_drivers.rename(columns= {'transect': 'transect_id' ,'r' : 'seas_ampl', 'p': 'seas_period', 't': 'time_recl', 'mcr': 'metric_recl',
                                         'dyjump': 'dy_recl', 'tfixed': 'recl_date', 'prob_cr': 'prob_recl',
                                         'type': 'ldb_type'})
df_drivers.head(2)

,transect_id,hotspot,p_mis,p_outl,seas_ampl,seas_period,prob_seas,prob_sandbar,prob_wrong,time_recl,metric_recl,dy_recl,recl_date,prob_recl,ldb_type,prob_ldb
0,BOX_209_524_188,prograding_feature_polygon_1,67.8,7.095344,87.0,365.0,0.095298,0.698573,0.206129,NaN,NaN,NaN,NaN,0.0,Downdrift,0.23799
1,BOX_209_524_189,prograding_feature_polygon_1,67.6,1.773836,20.0,365.0,0.704859,0.271206,0.023935,NaN,NaN,NaN,NaN,0.0,Downdrift,0.23799


In [5]:
df_transect = pd.read_csv('00. data/transect_system.csv')
df_drivers = df_transect.merge(df_drivers, how = 'left', on = 'transect_id')
df_drivers = df_drivers[[c for c in df_drivers.columns if not 'Unnamed' in c]]
df_drivers.head(2)

,transect_id,country_id,continent,country_name,changerate,flag_sandy,Start_lon,Start_lat,Intersect_lon,Intersect_lat,...,prob_seas,prob_sandbar,prob_wrong,time_recl,metric_recl,dy_recl,recl_date,prob_recl,ldb_type,prob_ldb
0,BOX_080_003_0,FRA,Europe,France,-91.82,True,55.308914,-21.229196,55.303230,-21.235681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BOX_080_003_1,FRA,Europe,France,-0.35,False,55.312392,-21.231845,55.306709,-21.238329,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_drivers = df_drivers[pd.notna(df_drivers['hotspot'])]

In [7]:
len(df_drivers), len(df_transect)

(1447, 17819)

#### Seasonality

In [8]:
df_seas = df_drivers[['transect_id', 'country_id', 'country_name', 'continent', 'country_name', 'flag_sandy', 'Intersect_lat', "Intersect_lon", "prob_seas", "prob_sandbar", "prob_wrong", "seas_ampl", 'seas_period']]
df_seas['seas_displ'] = 2*df_seas['seas_ampl']
df_seas.head(2)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_15184\496475293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seas['seas_displ'] = 2*df_seas['seas_ampl']


,transect_id,country_id,country_name,continent,country_name,flag_sandy,Intersect_lat,Intersect_lon,prob_seas,prob_sandbar,prob_wrong,seas_ampl,seas_period,seas_displ
2924,BOX_164_229_32,FRA,France,Europe,France,True,45.790300,-1.220615,0.0,0.0,1.0,10.0,183.0,20.0
2925,BOX_164_229_33,FRA,France,Europe,France,True,45.789026,-1.224710,0.0,0.0,1.0,14.0,89.0,28.0


In [9]:
def seasonal_classification(prob_seas):
    if  0.6 <= prob_seas <= 1:
        return 1
    if 0.4 <= prob_seas <= 0.6:
        return 2
    else:
        return 3
    
df_seas['seasonal_class'] = df_seas.apply(lambda x: seasonal_classification(x.prob_seas), axis = 1)
df_seas.head(2)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_15184\446795852.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seas['seasonal_class'] = df_seas.apply(lambda x: seasonal_classification(x.prob_seas), axis = 1)


,transect_id,country_id,country_name,continent,country_name,flag_sandy,Intersect_lat,Intersect_lon,prob_seas,prob_sandbar,prob_wrong,seas_ampl,seas_period,seas_displ,seasonal_class
2924,BOX_164_229_32,FRA,France,Europe,France,True,45.790300,-1.220615,0.0,0.0,1.0,10.0,183.0,20.0,3
2925,BOX_164_229_33,FRA,France,Europe,France,True,45.789026,-1.224710,0.0,0.0,1.0,14.0,89.0,28.0,3


In [19]:
df_seas['seasonal_class'] = pd.Categorical(df_seas['seasonal_class'], categories=[1, 2, 3])
fig = px.scatter_mapbox(df_seas, lat="Intersect_lat", 
                        lon="Intersect_lon", 
                        hover_name="transect_id", 
                        color= 'seasonal_class',
                        color_discrete_map = {1: 'green', 2: 'orange', 3: 'red'},
                        hover_data= ['seas_displ', 'seas_period'],
                        labels = {'country_name' : 'country', 'Intersect_lat': 'latitude', 'Intersect_lon': 'longitude', 'seas_displ': 'displacement', 'seas_period': 'period'},
                        zoom= 12, center = {'lat': 52.05, 'lon': 4.18})
fig.update_layout(mapbox_style="open-street-map", margin=dict(l=0,r=0,b=0,t=0), paper_bgcolor="white", showlegend=False)
fig.update_traces(marker={'size': 5})

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_15184\3365484659.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
df_sedimenttype = pd.read_csv('00. data/westeurope_beachtype.csv')
df_sedimenttype = df_sedimenttype[['transect_id', 'prediction_label']]
df_sedimenttype.head(2)


,transect_id,prediction_label
0,BOX_080_003_0,coastal_cliff
1,BOX_080_003_1,coastal_cliff


In [12]:
df_drivers = df_drivers.merge(df_sedimenttype, how = 'left', on = 'transect_id')

df_drivers.head(2)

,transect_id,country_id,continent,country_name,changerate,flag_sandy,Start_lon,Start_lat,Intersect_lon,Intersect_lat,...,prob_sandbar,prob_wrong,time_recl,metric_recl,dy_recl,recl_date,prob_recl,ldb_type,prob_ldb,prediction_label
0,BOX_164_229_32,FRA,Europe,France,-6.66,True,-1.217355,45.784463,-1.220615,45.790300,...,0.0,1.0,NaN,NaN,NaN,NaN,NaN,Downdrift,0.0,sand
1,BOX_164_229_33,FRA,Europe,France,-4.52,True,-1.220513,45.783487,-1.224710,45.789026,...,0.0,1.0,4.589041,0.94688,112.600081,2017.0,0.0,Downdrift,0.0,sand


In [13]:
# df_seas = df_drivers[df_drivers['prob_seas'] >= 0.7]
# df_sandbar = df_drivers[df_drivers['prob_sandbar'] >= 0.7]
# df_wrong = df_drivers[df_drivers['prob_wrong'] >= 0.7]

# perc_seas_sand = (len(df_seas[df_seas['prediction_label'] == 'sand']) /len(df_seas))
# perc_tot_sand = (len(df_drivers[df_drivers['prediction_label'] == 'sand']) /len(df_drivers))
# perc_wrong_sand = (len(df_wrong[df_wrong['prediction_label'] == 'sand']) /len(df_wrong))

# print(perc_seas_sand)
# print(perc_tot_sand)
# print(perc_wrong_sand)